# Git — 4-hour crash notes
A simple, practical summary of the Git concepts/commands I learned in one long session. This is **not** a complete Git book — it’s meant to be quick to revisit.

## In short: what to know
- When stuck: `git help <command>` (offline manual). Example: `git help commit` or `git commit -h`.
- Git tracks **snapshots** via commits (not just “diffs”).
- Three core areas: **Working tree** → **Staging area (index)** → **Repository (HEAD)**.
- Common flow: `status` → `add` → `commit` → `push`.
- You can almost always recover using `git reflog` (as long as the repo exists).

## Quick list

This is the “one-liner command memory” style I started with:

- `git status` — see what’s changed (working tree / staged / untracked).
- `git add <file>` — stage changes.
- `git commit -m "msg"` — save a snapshot into history.
- `git log` — view commit history (`--oneline --graph --decorate` helps a lot).
- `git branch` — list/create branches.
- `git switch <branch>` — change branches (older: `git checkout`).
- `git push <remote> <branch>` — upload commits to a remote (e.g., GitHub).
- `git pull` — bring remote changes down (fetch + merge/rebase).
- `git merge` — integrate another branch.
- `git rebase` — replay commits onto a new base (rewrites history).
- `git reset` — move `HEAD` (soft/mixed/hard).
- `git revert` — undo a commit safely by creating a new commit.
- `git stash` — temporarily put changes aside.
- `git cherry-pick <commit>` — take one commit from somewhere else.
- `git bisect` — find which commit introduced a bug (binary search).
- `git tag` — mark a version/release.
- `git reflog` — “life saver” log of where `HEAD` was (great for recovery).

## Mental model (the part that makes Git click)

**Three areas:**
- **Working tree**: your files on disk
- **Staging area (index)**: what will go into the next commit
- **Repository (HEAD)**: committed history

**Common file states you’ll see in `git status`:**
- **Untracked**: Git isn’t tracking the file yet
- **Modified**: tracked file changed in working tree
- **Staged**: changes added to index (ready to commit)

Think: **edit → stage → commit**.

From my original note (how I remembered it): “Commit - Staged - Untracked”.
- The corrected version is: **Working tree (modified/untracked)** → **Staged** → **Committed (HEAD)**.

## Setup & config (only the essentials)

Initialize a repo (in an empty folder):
```bash
git init
```
From my notes: `git init` creates a hidden `.git/` folder that stores the repo metadata/history.

Set name/email (most common global settings):
```bash
git config --global user.name "Your Name"
git config --global user.email "you@example.com"
```

Config levels (priority is top → bottom):
- `worktree` → `local` (repo) → `global` → `system`
- Lower in the list is more general; higher overrides it (example: **local overrides global**).

## Everyday commands (90% of usage)

Check what changed:
```bash
git status
git diff
git diff --staged
```

Stage + commit:
```bash
git add <file>        # or: git add .
git commit -m "message"
```

View history (nice defaults):
```bash
git log --oneline --decorate --graph -n 20
```
From my notes: commits show an ID (often SHA-1 in most repos), message, author/date.

Small quality-of-life options I keep using:
- `-n <num>` to limit output
- `--no-pager` if you don’t want the pager
- You can do `git log <branch>` to target a branch

Note: Git has “porcelain” (high-level) commands you use daily; “plumbing” exists, but you can ignore it at first.

## Branching (create / switch / delete)

Create a branch and switch to it:
```bash
git switch -c my-branch
```

Switch branches:
```bash
git switch main
```

List / delete:
```bash
git branch            # list
git branch -d my-branch   # safe delete (merged)
git branch -D my-branch   # force delete
```

Notes:
- `git switch` is the modern “just switch branches” command.
- `git checkout` is older and does more things; you’ll still see it in tutorials.

## Merge vs rebase (simple rule)

Merge brings another branch in **without rewriting history**:
```bash
git merge feature
```

Rebase replays your commits on a new base (**rewrites history**):
```bash
git rebase main
```

Simple rule I follow:
- **Local branch only** (not shared yet) → rebase is OK for a clean line
- **Already pushed / shared** → prefer merge (or only rebase if your team explicitly does it)

## Remotes (GitHub, origin, upstream)

A **remote** is just a named URL to another repo.
```bash
git remote -v
git remote add origin <url>
```

Typical flow:
```bash
git fetch             # download remote refs (no merge)
git pull              # fetch + merge (or rebase, depending config)
git push -u origin my-branch
```

From my notes: Git is **distributed version control** — everyone can have the full history locally.

Git vs GitHub:
- **Git** = tool + local history format
- **GitHub** = hosting/collaboration platform that uses Git repos

Forking tip (common on GitHub):
- Your fork remote is usually `origin`
- The original project is often added as `upstream`

## Undo / recover (the important ones)

Undo changes in working tree (newer command):
```bash
git restore <file>
```

Unstage (keep file changes):
```bash
git restore --staged <file>
```

Reset modes (moves `HEAD`; can also update index/working tree):
- `--soft`: move `HEAD`, keep index + working tree
- `--mixed` (default): move `HEAD`, reset index, keep working tree
- `--hard`: move `HEAD`, reset index + working tree (destructive)

Undo a *commit* safely (no history rewrite):
```bash
git revert <commit>
```

If you messed up:
```bash
git reflog
```

## Conflicts (merge / rebase)

When Git can’t auto-merge changes, you’ll see conflict markers in files. The normal fix loop:
1. Open conflicted files and choose what the final content should be
2. `git add <file>` after each file is resolved
3. Finish the operation: commit the merge, or continue the rebase

Helpful during conflict resolution:
```bash
git status
git diff
```

Notes on **ours/theirs**:
- In a merge conflict, “ours” = your current branch; “theirs” = the branch being merged in.
- In a rebase conflict, it can feel flipped depending on perspective—use `git status` and read carefully.

From my notes (team reality): if it’s messy, you can also abort and restart carefully:
```bash
git merge --abort
git rebase --abort
```

## Useful extras (when you start leveling up)

Stash (temporary shelf):
```bash
git stash
git stash pop
```

Cherry-pick (take 1 commit from elsewhere):
```bash
git cherry-pick <commit>
```

Bisect (find the commit that introduced a bug):
```bash
git bisect start
```

Squash (combine commits):
- Usually done with interactive rebase: `git rebase -i <base>`
- Or via merge: `git merge --squash feature`

Tags (mark releases / versions):
```bash
git tag v1.0.0
```

## Team habits (simple + realistic)

- Prefer working on feature branches + PRs; keep `main` protected/stable (don’t commit straight to it in teams).
- Write commit messages that explain *why* (not just what). Example: `fix: handle empty input`.
- If your team uses issue numbers, include them in commit/PR titles (format depends on the team/tool). Example I used: `#1 - message`.
- Use `.gitignore` for:
  - environment/local files
  - secrets/sensitive data (API keys, passwords)
  - build outputs, logs, OS junk
- Forking (GitHub feature): fork → clone → branch → push → open PR.